## Import Packages

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline


## Read File

In [2]:
data = pd.read_csv('OnlineNewsPopularity.csv')
data.columns = [col.strip() for col in data.columns]


## Create isolation forest model to remove outliers
[API Link to IF](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html#sklearn.ensemble.IsolationForest)

In [3]:
old_data_length = len(data)
IF_model = IsolationForest(contamination=0.1) # Remove 10% data
outliers = IF_model.fit_predict(data.drop(columns=['url', 'timedelta', 'shares'])) # Delete columns than don't need in IF and train IF model
data['outliers'] = outliers

# Remove outliers
data_outliers = data[data['outliers'] == -1]
new_data = data[data['outliers'] != -1].drop(columns = ['outliers'])

new_data_len = len(new_data)

print(f"There are total {old_data_length - new_data_len} rows been removied.")


/Users/will./anaconda3/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


There are total 3965 rows been removied.


## Creat a function to classify an article to different categories
* Great Article: Shares >= 90%
* Good Article: 70% <= Shares < 90%
* Normal Article: 30% <= Shares < 70%
* Bad article: Shares < 30%

In [4]:
def article_classifier(shares, top_percent):
    if shares >= top_percent[0.9]:
        return 'Great'
    elif shares >= top_percent[0.7]:
        return 'Good'
    elif shares >= top_percent[0.3]:
        return 'Normal'
    else:
        return 'Bad'
        

## Data Classifier

In [5]:
top_percent = new_data['shares'].quantile([0.3, 0.7, 0.9])

quality = []
for index, row in new_data.iterrows():
    quality.append(article_classifier(row['shares'], top_percent))

new_data['quality'] = quality


## Split dataset into x and y and one-hot encode quality column

In [6]:
x = new_data.drop(columns=['quality', 'url', 'timedelta', 'shares'], axis=1)
y = OneHotEncoder().fit_transform(new_data[['quality']]).toarray()


## Train test split

In [7]:

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=10)


## Scale the data to Improve the speed of training SVM Linear Model

In [8]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
scale_xTrain = standard_scaler.fit_transform(xTrain)
scale_xTest = standard_scaler.transform(xTest)

## Create and train model(NN)

In [21]:
# enable multiple GPU when using the mirror_strategy

def build_nn_model(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

mirror_strategy = tf.distribute.MirroredStrategy()
with mirror_strategy.scope():
    nn_model1 = build_nn_model(scale_xTrain.shape[1:], yTrain.shape[1])
    nn_model2 = build_nn_model(scale_xTrain.shape[1:], yTrain.shape[1])



    nn_model1.fit(scale_xTrain, yTrain, epochs=10)
    nn_model2.fit(scale_xTrain, yTrain, epochs=10)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/10
892/892 [==============================] - 1s 543us/step - loss: 1.2062 - accuracy: 0.4386
Epoch 2/10
892/892 [==============================] - 0s 553us/step - loss: 1.1718 - accuracy: 0.4543
Epoch 3/10
892/892 [==============================] - 1s 577us/step - loss: 1.1587 - accuracy: 0.4664
Epoch 4/10
892/892 [==============================] - 1s 564us/step - loss: 1.1451 - accuracy: 0.4741
Epoch 5/10
892/892 [==============================] - 0s 558us/step - loss: 1.1357 - accuracy: 0.4813
Epoch 6/10
892/892 [==============================] - 0s 542us/step - loss: 1.1231 - accuracy: 0.4847
Epoch 7/10
892/892 [==============================] - 0s 541us/step - loss: 1.1128 - accuracy: 0.4954
Epoch 8/10
892/892 [==============================] - 0s 558us/step - loss: 1.0993 - accuracy: 0.5013
Epoch 9/10
892/892 [==============================] - 0s 555us/step - loss: 1.0848

## Create and train model(Random Forest)

In [10]:
RF_model = RandomForestClassifier(n_estimators=100)
RF_model.fit(scale_xTrain, yTrain.argmax(axis=1))


RandomForestClassifier()

## Make predictions with each model individually

In [11]:
nn1_pred = nn_model1.predict(scale_xTest)
nn2_pred = nn_model2.predict(scale_xTest)
RF_pred = tf.keras.utils.to_categorical(RF_model.predict(scale_xTest), num_classes=yTrain.shape[1])

223/223 [==============================] - 0s 242us/step


## Create ensemble model

In [12]:
ensemble_pred = (nn1_pred + nn2_pred + RF_pred) / 3
ensemble_labels = tf.argmax(ensemble_pred, axis=1).numpy()

## Model Evaluation

In [13]:
print(classification_report(yTest.argmax(axis=1), ensemble_labels))


              precision    recall  f1-score   support

           0       0.50      0.48      0.49      2033
           1       0.39      0.15      0.22      1432
           2       0.36      0.06      0.10       721
           3       0.46      0.70      0.55      2950

    accuracy                           0.46      7136
   macro avg       0.43      0.35      0.34      7136
weighted avg       0.45      0.46      0.42      7136

